In [9]:
from flask import (
    Flask,
    render_template,
    jsonify,
    request,
    redirect)

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine

app = Flask(__name__)

engine = create_engine("sqlite:///db/belly_button_biodiversity.sqlite")
Base = automap_base()
Base.prepare(engine, reflect=True)

OTU = Base.classes.otu
samp_meta = Base.classes.samples_metadata
Samples = Base.classes.samples

session = Session(engine)

In [20]:
x = session.query(Samples)
print(x)

SELECT samples.otu_id AS samples_otu_id, samples."BB_940" AS "samples_BB_940", samples."BB_941" AS "samples_BB_941", samples."BB_943" AS "samples_BB_943", samples."BB_944" AS "samples_BB_944", samples."BB_945" AS "samples_BB_945", samples."BB_946" AS "samples_BB_946", samples."BB_947" AS "samples_BB_947", samples."BB_948" AS "samples_BB_948", samples."BB_949" AS "samples_BB_949", samples."BB_950" AS "samples_BB_950", samples."BB_952" AS "samples_BB_952", samples."BB_953" AS "samples_BB_953", samples."BB_954" AS "samples_BB_954", samples."BB_955" AS "samples_BB_955", samples."BB_956" AS "samples_BB_956", samples."BB_958" AS "samples_BB_958", samples."BB_959" AS "samples_BB_959", samples."BB_960" AS "samples_BB_960", samples."BB_961" AS "samples_BB_961", samples."BB_962" AS "samples_BB_962", samples."BB_963" AS "samples_BB_963", samples."BB_964" AS "samples_BB_964", samples."BB_966" AS "samples_BB_966", samples."BB_967" AS "samples_BB_967", samples."BB_968" AS "samples_BB_968", samples."

In [10]:
@app.route("/")
def home():
    """Render Home Page."""
    return render_template("index.html")

In [11]:
@app.route("/names")
def sample_names():
    results = session.query(samp_meta.SAMPLEID).all()
    names = []
    for name in results:
        names.append("BB_" + str(name[0]))  
    return jsonify(names)




In [12]:
@app.route("/otu")
def get_otu_descriptions():
    results = session.query(OTU.lowest_taxonomic_unit_found).all()
    otu_descriptions = []
    i = 0
    while i < len(results):
        otu_descriptions.append(results[i][0])
        i += 1

    return jsonify(otu_descriptions)




In [13]:
@app.route("/metadata/<sample>")
def get_sample_metadata(sample):
    sample = int(sample[3:])
    results = session.query(samp_meta.AGE, samp_meta.BBTYPE, samp_meta.ETHNICITY, samp_meta.GENDER, samp_meta.LOCATION, samp_meta.SAMPLEID).filter(samp_meta.SAMPLEID == sample)
    results_dict = {"AGE": results[0][0], "BBTYPE": results[0][1], "ETHNICITY": results[0][2], "GENDER": results[0][3], "LOCATION": results[0][4], "SAMPLEID": results[0][5]}
    return jsonify(results_dict)



In [14]:
@app.route("/wfreq/<sample>")
def get_washing_freq(sample):
    sample = int(sample[3:])
    results = session.query(samp_meta.WFREQ).filter(samp_meta.SAMPLEID == sample)    
    washing_freq = int(results[0][0])
    return jsonify(washing_freq)
    


In [15]:
@app.route("/samples/<sample>")
def get_sample_values(sample):
    results = session.query(Samples.otu_id, getattr(Samples, sample)).order_by(getattr(Samples, sample).desc())
    otu_ids = []
    sample_values = []

    for item, value in results:
        otu_ids.append(item)
        sample_values.append(value)

    results_dict = {"otu_ids": otu_ids, "sample_values": sample_values}
    return jsonify(results_dict)

In [17]:
if __name__ == '__main__':
    app.run(debug=True)

UnsupportedOperation: not writable